In [3]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.optimize import minimize

# Load test samples
from aeons.tools import pickle_in
samples_g_1, samples_c_1, samples_w_1 = pickle_in("../test_samples/samples_1.pickle")

from aeons.covariance import logX_mu, logX_covinv_chol, points_at_iteration, X_mu, X_Sigmainv, data_at_iteration
from aeons.true_distribution import generate_Xs
from aeons.lm_partial import analytic_lm_params
from bayes import logPr_bayes, logPr_gaussian, minimise_ls, minimise_bayes, minimise_gaussian
from likelihoods import likelihood

### Try on real likelihood model

In [2]:
def full_like():
    def func(X, theta):
        logLmax, d, sigma = theta
        return logLmax - X**(2/d)/(2*sigma**2)

    def inverse(logL, theta):
        logLmax, d, sigma = theta
        return (2*sigma**2 * (logLmax - logL))**(d/2)

    def prime(X, theta):
        logLmax, d, sigma = theta
        return - 1/(d*sigma**2) * X**(2/d - 1)

    return likelihood(func, inverse, prime)

full = full_like()

Check if the loss function diagnostic meets the eye test

In [4]:
def loss_theta(theta_true, theta_predicted):
    frac_diffs = (theta_true - theta_predicted)/theta_true
    return np.sum(frac_diffs**2)

In [6]:
nk, logL = data_at_iteration(samples_g_1, 100)
mean_X = X_mu(nk)
covinv_X = X_Sigmainv(nk)
covinv_ls = np.eye(len(mean_X))

In [9]:
b = 0
l = 0
theta_true = np.array([1, 10, 0.1])
for i in range(50):
    Xs = generate_Xs(nk)
    y = full.func(Xs, theta_true)
    theta_b =  minimise_bayes(y, full, mean_X, covinv_X, theta_true).x
    theta_l = minimise_ls(y, full, mean_X, theta_true).x
    if loss_theta(theta_true, theta_b) < loss_theta(theta_true, theta_l):
        b += 1
    else:
        l += 1
print('b', b)
print('l', l)

b 22
l 28


In [ ]:
theta_l = analytic_lm_params(logL, mean_X, 10)
theta_b = minimise_bayes_full(logL, mean_X, covinv_X, theta_l).x
print(theta_b, theta_l)

[-0.19023726  9.76205936  0.10020569] [-0.3088430612631896, 9.72222790754946, 0.10080065717795862]


In [ ]:
logPr_bayes_full(logL, mean_X, covinv_X, theta_b)

-6243.92316118433